In [ ]:
imu_dic = {key: imu_static_dic[key] for key in imu_static_dic.files}

xl1x = imu_dic['Node[1].XL1.X']
xl1x_mean = np.mean(xl1x)
offset = 0.0015
mask_x = (xl1x < xl1x_mean + offset) & (xl1x > xl1x_mean - offset)
xlx_clip = xl1x[mask_x]

xly = imu_dic['Node[1].XL1.Y']
xly_mean = np.mean(xly)
offset = 0.0015
mask_y = (xly < xly_mean + offset) & (xly > xly_mean - offset)
xly_clip = xly[mask_y]

xlz = imu_dic['Node[1].XL1.Z']
xlz_mean = np.mean(xlz)
offset = 0.0015
mask_z = (xlz < xlz_mean + offset) & (xlz > xlz_mean - offset)
xlz_clip = xlz[mask_z]

print(imu_dic['Node[1].XL1.X'].shape, xlx_clip.shape, imu_dic['Node[1].XL1.Y'].shape, xly_clip.shape, imu_dic['Node[1].XL1.Z'].shape, xlz_clip.shape)

gy1x = imu_dic['Node[1].GY1.X']
gy1x_mean = np.mean(gy1x)
offset = 0.25
gy_mask_x = (gy1x < gy1x_mean + offset) & (gy1x > gy1x_mean - offset)
gyx_clip = gy1x[gy_mask_x]

gyy = imu_dic['Node[1].GY1.Y']
gyy_mean = np.mean(gyy)
offset = 0.25
gy_mask_y = (gyy < gyy_mean + offset) & (gyy > gyy_mean - offset)
gyy_clip = gyy[gy_mask_y]

gyz = imu_dic['Node[1].GY1.Z']
gyz_mean = np.mean(gyz)
offset = 0.25
gy_mask_z = (gyz < gyz_mean + offset) & (gyz > gyz_mean - offset)
gyz_clip = gyz[gy_mask_z]

print(imu_dic['Node[1].GY1.X'].shape, gyx_clip.shape, imu_dic['Node[1].GY1.Y'].shape, gyy_clip.shape, imu_dic['Node[1].GY1.Z'].shape, gyz_clip.shape)

In [ ]:
gy_mask_combined = np.logical_and(gy_mask_x, gy_mask_y, gy_mask_z)
gy_timestamps = imu_dic["Node[1].GY1.timestamp"]
new_gy_ts = gy_timestamps[gy_mask_combined]


xl_mask_combined = np.logical_and(mask_x, mask_y, mask_z)
timestamps = imu_dic["Node[1].XL1.timestamp"]
new_xl_ts = timestamps[xl_mask_combined]

print(new_xl_ts.shape)

In [ ]:
xl_stack = np.array([imu_dic['Node[1].XL1.interval_id'][xl_mask_combined],imu_dic['Node[1].XL1.timestamp'][xl_mask_combined], 
            imu_dic['Node[1].XL1.X'][xl_mask_combined],imu_dic['Node[1].XL1.Y'][xl_mask_combined],
              imu_dic['Node[1].XL1.Z'][xl_mask_combined]])

gy_stack = np.array([imu_dic['Node[1].GY1.interval_id'][gy_mask_combined],imu_dic['Node[1].GY1.timestamp'][gy_mask_combined], 
            imu_dic['Node[1].GY1.X'][gy_mask_combined],
            imu_dic['Node[1].GY1.Y'][gy_mask_combined], imu_dic['Node[1].GY1.Z'][gy_mask_combined]])

print(xl_stack.shape, gy_stack.shape)

In [ ]:
from scipy.signal import decimate, resample_poly, resample
from ahrs.filters.madgwick import Madgwick #ahrs library requires git clone, do not pip install

def preprocess_sensor(dat):
    """ 
    Sorts sensor data by timestamp and removes duplicates
    Params: 
        dat: sensor data 
    """
    dat = np.array(dat).astype(float)
    sorted_dat = dat[dat[:,0].argsort()]

    _, unique_indices = np.unique(sorted_dat[:,0], return_index = True)
    processed_data = sorted_dat[unique_indices]
    return processed_data

def clip_xl(xl_dat, clip):
    """
    XL sensors above certain thresholds are defective
    """
    xl_clipped = np.where(xl_dat > clip, clip, xl_dat)
    xl_clipped = np.where(xl_clipped < -1*clip, -1*clip, xl_clipped)
    return xl_clipped

def preprocess_imu(xl_raw, gyr_raw, gyr_offset):
    """
    correct units, remove gyr offset
    Returns:
        data_matrix: (xl, mag, gyr)
    """

    xl = xl_raw * 9.810665
    gyr = gyr_offset_radians(gyr_raw, gyr_offset)
    return xl, gyr

def gyr_offset_radians(gyr_dat, offset):
    """Put gy data in radians, remove measured error """
    gyr_off = gyr_dat - offset
    gyro_rad = gyr_off * ( np.pi / 180)
    return gyro_rad

def time_align(xyz, sensor_odr, interval_odr):
    """ 
    Upsample data 2x interval odr(fourier signal), decimate data back down(lowpass filter), interpolate to desired odr, select current interval points
    """
    xyz_w = []
    len_w = []
    for i, ts in enumerate(xyz):
        xyz_w.append(ts)
        len_w.append(len(ts))

    xyz_w = np.vstack(xyz_w)
    # if name == 'xl':
        # print('raw',xyz_w.shape)

    xyz_r = resample(xyz_w, num=(sensor_odr*3)*2, axis=0)
    factor = xyz_r.shape[0]//((sensor_odr*3))
    xyz_d = decimate(xyz_r, q=factor, axis=0)

    current_points = xyz_d[interval_odr:interval_odr*2]
    
    # print('final', np.array(current_points).shape)
    return np.array(current_points)

def normalize(q): 
    #norm = np.sqrt(q @ q.T)
    norm = magnitude(q)
    return q/norm 

def quatConjugate(q):
        w,x,y,z = q
        q_star = np.array([w,-x,-y,-z])
        return q_star
        
def quatProduct(q, p):
        q = np.array(q)
        p = np.array(p)
        w, x, y, z = list(q)

        q_matrix = np.array([
            [w, -x, -y, -z],
            [x, w, -z, y],
            [y, z, w, -x],
            [z, -y, x, w]
        ])
        return q_matrix @ p

def inverse(q):
    q_star = quatConjugate(q)
    mag = magnitude(q)
    if  0.998 < mag < 1.001:
        return q_star
    else:
        q_inverse = q_star / mag**2
        return q_inverse
    
def magnitude(q):
    return np.sqrt(q[0]**2 + q[1]**2 + q[2]**2 + q[3]**2)

def rotate_vector(q, v, active=True):
    v = np.insert(np.array(v), 0, 0)  # convert to pure quaternion
    q_inv = inverse(q)

    if active: # object rotation
        q_product = quatProduct(q, v)
        q_rot = quatProduct(q_product, q_inv)
    else: # coordinate rotation
        q_product = quatProduct(q_inv, v)
        q_rot = quatProduct(q_product, q)

    return q_rot[1:]

def R(q_w, q_x, q_y, q_z):
    """
    Direction Cosine Matrix to rotate between two frames
    """
    # rotation_matrix = np.array([
    #     [q_w**2+q_x**2-q_y**2-q_z**2, 2*(q_x*q_y-q_w*q_z), 2*(q_x*q_z+q_w*q_y)],
    #     [2*(q_x*q_y+q_w*q_z), q_w**2-q_x**2+q_y**2-q_z**2, 2*(q_y*q_z-q_w*q_x)],
    #     [2*(q_x*q_z-q_w*q_y), 2*(q_w*q_x+q_y*q_z), q_w**2-q_x**2-q_y**2+q_z**2]
    # ])
            
    rotation_matrix = np.array([
        [1.0-2.0*(q_y**2+q_z**2), 2.0*(q_x*q_y-q_w*q_z), 2.0*(q_x*q_z+q_w*q_y)],
        [2.0*(q_x*q_y+q_w*q_z), 1.0-2.0*(q_x**2+q_z**2), 2.0*(q_y*q_z-q_w*q_x)],
        [2.0*(q_x*q_z-q_w*q_y), 2.0*(q_w*q_x+q_y*q_z), 1.0-2.0*(q_x**2+q_y**2)]
        ])
    return rotation_matrix

def rotate_to_ned(q,vec):
    """
    rotates raw data by quaternion into global frame
    """
    rotation_matrix = R(*q)
    global_estimate = rotation_matrix.T @ vec  #eq. 10 
    
    return global_estimate

def align_timestamps(xl, gy, interval_odr, gyr_offset):
    cols =['interval','timestamp','x', 'y', 'z']
    xl_df = pd.DataFrame(xl, columns=cols)
    xl_group = xl_df.groupby('interval')

    gy_df = pd.DataFrame(gy, columns=cols)
    gy_group = gy_df.groupby('interval')

    xl_global = []
    prev_q = [1,0,0,0]

    xl_w = [] #(3, n, 3) history of 3, n data , xyz 
    gy_w = []
    mg_w = []

    for (name1, xl_grp), (name2, gy_grp),in zip(gy_group, xl_group):
        if name1 != name2:
            continue

        # ts1 = xl_grp.iloc[xl_grp.shape[0]//2]['timestamp']
        # interval_id = xl_grp.iloc[0]['interval']

        xl_1 = preprocess_sensor(xl_grp.drop(columns=['interval']).values)
        xl1_clipped = clip_xl(xl_1[:,1:], 2)
        gy_1 = preprocess_sensor(gy_grp.drop(columns=['interval']).values) #remove unique values 

        xl_w.append(xl1_clipped)
        gy_w.append(gy_1[:,1:])

        if len(xl_w) > 3: 
            xl_w.pop(0)
            gy_w.pop(0)

        if len(xl_w) != 3:
            continue 

        xl_t = time_align(xl_w, 1000, interval_odr)
        gy_t = time_align(gy_w, 1000, interval_odr)

        xl_p, gy_p = preprocess_imu(xl_t, gy_t, gyr_offset)

        model = Madgwick(acc=xl_p, gyr=gy_p, dt=1/interval_odr, q0 = prev_q)
        prev_q = model.Q[-1]
        interval_q = model.Q

        group_xl_ned = []
        for q,x in zip(np.array(interval_q), xl_t):
            x*=9.8
            mag = np.linalg.norm(x)
            norm_vec = x / mag
            ned_vec_1 = rotate_to_ned(q, norm_vec)
            vec1 = np.round(ned_vec_1,4)

            q = normalize(q)
            ned_vec_2 = rotate_vector(q, norm_vec, active=False)
            vec2 = np.round(ned_vec_2,4)

            assert np.allclose(vec1, vec2) , f"Not equal vec1: {vec1}, vec2: {vec2}"

            ned_vec_1 *= mag
            ned_vec_1[2] -= 9.810665
            # xl_global.append(ned_vec_1)
            group_xl_ned.append(ned_vec_1)
        
        group_ts = [xl_grp.iloc[xl_grp.shape[0]//2]['timestamp'] for _ in range(np.array(group_xl_ned).shape[0])]
        group_interval = [xl_grp.iloc[xl_grp.shape[0]//2]['interval'] for _ in range(np.array(group_xl_ned).shape[0])]

        group_xl_ned = np.column_stack([group_xl_ned, group_ts, group_interval])
        xl_global.append(group_xl_ned)

    return xl_global

In [ ]:
xl_stack = np.array([imu_dic['Node[1].XL1.interval_id'][xl_mask_combined],imu_dic['Node[1].XL1.timestamp'][xl_mask_combined], 
            imu_dic['Node[1].XL1.X'][xl_mask_combined],imu_dic['Node[1].XL1.Y'][xl_mask_combined],
              imu_dic['Node[1].XL1.Z'][xl_mask_combined]])

gy_stack = np.array([imu_dic['Node[1].GY1.interval_id'][gy_mask_combined],imu_dic['Node[1].GY1.timestamp'][gy_mask_combined], 
            imu_dic['Node[1].GY1.X'][gy_mask_combined],
            imu_dic['Node[1].GY1.Y'][gy_mask_combined], imu_dic['Node[1].GY1.Z'][gy_mask_combined]])

print(xl_stack.shape, gy_stack.shape)

In [ ]:
offset = np.array([.58, .42, -.3])
xl_global = align_timestamps(xl_stack.T, gy_stack.T, 1000, offset)